In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 

In [4]:
data = pd.read_csv("merged.csv")
data.head()

,Unnamed: 0,text,truefalse
0,1147,WASHINGTON (Reuters) - Republican U.S. Represe...,1
1,9603,"In September of 2016, 3 separate reports of ra...",0
2,21294,WARSAW (Reuters) - Poland said on Monday that ...,1
3,14098,BRUSSELS (Reuters) - When Theresa May visits B...,1
4,4350,If there s one thing the Donald Trump is good ...,0


In [5]:
import nltk 
nltk.download("stopwords")
from nltk.corpus import stopwords, wordnet
stopwords.words("english")
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yash0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yash0\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yash0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
import re
import string
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text
data["text"] = data["text"].apply(wordopt)

In [9]:
data.head()

,Unnamed: 0,text,truefalse
0,1147,washington reuters republican u s represe...,1
1,9603,in september of separate reports of racist ...,0
2,21294,warsaw reuters poland said on monday that ...,1
3,14098,brussels reuters when theresa may visits b...,1
4,4350,if there s one thing the donald trump is good ...,0


In [7]:
from sklearn.model_selection import train_test_split
X = data["text"]
y = data["truefalse"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,  random_state=42)

In [12]:
vect = TfidfVectorizer(stop_words="english")
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

In [13]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [14]:
y_predxg = xg.predict(X_test)

In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predxg))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4657
           1       1.00      1.00      1.00      4323

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

